# Estimating Macro Vars

In [298]:
# Load Pkgs
using DataFrames;
using CSV;
using Optim;
using Clp
using Ipopt;
using JuMP;

In [137]:
# Set Dirs
dir = "/home/justin/Dropbox/projects/julia/est_macro_vars_practice/";

# Read Data
df = CSV.File(dir * "data_agg_econ.csv") |> DataFrame;

#### 2.1

In [139]:
function compute_tup(df, k, year)
    
    T = sum(df[df[:year] .<= year, :].exprt_chng[1:k[1]] .> 0) > k[1] / 2
    U = sum(df[df[:year] .<= year, :].unemp_chng[1:k[2]] .> 0) > k[2] / 2
    P = sum(df[df[:year] .<= year, :].prices_chng[1:k[3]] .> 0) > k[3] / 2
    
    return [T, U, P]
end;

In [140]:
k = [8,4,6]
year = 2012
chngs_846 = compute_tup(df, k, year)
chngs_846

3-element Array{Bool,1}:
  true
 false
  true

#### 2.2

In [360]:
X

1×3 LinearAlgebra.Adjoint{Bool,Array{Bool,1}}:
 true  false  true

In [363]:
function find_optim_gamma(G, X, init_gamma=[0.5, 0.5, 0.5])
    
    m = Model(with_optimizer(Ipopt.Optimizer), )

    M = size(X)[1]
    N = size(X)[2]
    gamma = init_gamma
    
    @variable(m, 1 >= gamma[1:M,1:N] >= 0 );
    @constraint(m, con_1, sum(X'*gamma) >= 0)
    @constraint(m, con_2, sum(-X'*gamma) >= -1)
    @NLobjective(m, Min, sum(((X[i]*gamma[i]) - G[1])^2 for i in 1:N))
    JuMP.optimize!(m);
    
    print(m)
    
    return [value(gamma[i]) for i in 1:3]
end;

In [364]:
G = df[df[:year] .== 2013, :].gdp_chng
X = chngs_846'

find_optim_gamma(G, X)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        3
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        2
        inequality constraints with only lower bounds:        2
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

3-element Array{Float64,1}:
 0.250000006987959 
 0.0               
 0.2500000069879589

In [330]:
X

1×3 LinearAlgebra.Adjoint{Bool,Array{Bool,1}}:
 true  false  true

In [331]:
G

1-element Array{Union{Missing, Float64},1}:
 2.7